# Stock Price Prediction

## Time Series Plots

In [48]:
import quandl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [175]:
url = "https://www.quandl.com/api/v3/datasets/WIKI/AAPL.csv"
df = pd.read_csv(url)

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9400 entries, 0 to 9399
Data columns (total 13 columns):
Date           9400 non-null object
Open           9400 non-null float64
High           9400 non-null float64
Low            9400 non-null float64
Close          9400 non-null float64
Volume         9400 non-null float64
Ex-Dividend    9400 non-null float64
Split Ratio    9400 non-null float64
Adj. Open      9400 non-null float64
Adj. High      9400 non-null float64
Adj. Low       9400 non-null float64
Adj. Close     9400 non-null float64
Adj. Volume    9400 non-null float64
dtypes: float64(12), object(1)
memory usage: 954.8+ KB


In [177]:
# Lets convert the dataframe index into the date series
#df = df.iloc[::-1]
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df = df.sort_values(by=['Date'], ascending=[True])
df.set_index('Date', inplace=True)

## Fill in the Empty data points using forward fill

In [179]:
df = df.resample('D').ffill().reset_index()
#df = df.sort_values(by = df['Date'],ascending = True)

In [182]:
df.head(7)

,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0
1,1980-12-13,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0
2,1980-12-14,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0
3,1980-12-15,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.402563,0.402563,0.400652,0.400652,43971200.0
4,1980-12-16,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.373010,0.373010,0.371246,0.371246,26432000.0
5,1980-12-17,25.87,26.00,25.87,25.87,385900.0,0.0,1.0,0.380362,0.382273,0.380362,0.380362,21610400.0
6,1980-12-18,26.63,26.75,26.63,26.63,327900.0,0.0,1.0,0.391536,0.393300,0.391536,0.391536,18362400.0


In [185]:
# Time series decompostion

import statsmodels.api as sm
res = sm.tsa.seasonal_decompose(df.Volume.values)
#resplot = res.plot()

ValueError: You must specify a freq or x must be a pandas object with a timeseries index with a freq not set to None